In [2]:
using JuMP,Gurobi

In [3]:
using CSV
data = CSV.read("data_transfered.csv")

,ID,Country or area,Country code,Capital City
,Int64,String,Int64,String
1,1,Afghanistan,4,Kabul
2,2,Albania,8,Tiran\x91 (Tirana)
3,3,Algeria,12,El Djaza\x95r (Algiers)
4,4,American Samoa,16,Pago Pago
5,5,Andorra,20,Andorra la Vella
6,6,Angola,24,Luanda
7,7,Anguilla,660,The Valley
8,8,Antigua and Barbuda,28,St. John's
9,9,Argentina,32,Buenos Aires


In [4]:
I = [1:241;]
J = [1:241;]
de = 110.25

110.25

In [5]:
c = zeros((241,241))
for i = 1:241
    for j = 1:241
        c[i,j] = de*((data[i,8]-data[j,8])^2 + ((data[i,9]-data[j,9])*cos(data[j,8]))^2)^(1/2)
    end
end

In [8]:
p = 10
K = [112 103 139 70 47 17 9 168 64 177]
Model_modify = Model(Gurobi.Optimizer)
@variable(Model_modify,x[i in I],Bin) # if city i is distribution center
@variable(Model_modify,y[i in I,j in J],Bin) #if city i is served by city j
@variable(Model_modify,C,Bin) #if there's more than 2 distributio centers in the first 20 cities
@objective(Model_modify,Min,sum(y[i,j]*c[i,j]*data[i,10] for i in I , j in J))
@constraints(Model_modify,begin
        [i in I], sum(y[i,j] for j in J) == 1
        [i in I , j in J], y[i,j]-x[j] <= 0
        sum(x[i] for i in I) == p
        # modification 1
        sum(x[i] for i in 1:20) <= 21*sum(x[i] for i in 30:40)
        # modification 2
        sum(x[i] for i in 1:20) >= 3C
        sum(x[i] for i in 1:20) <= 2+18C
        sum(x[i] for i in 30:40) >= C
        sum(x[i] for i in 30:40) <= 10C
        # modification 3
        x[39] + x[230] <= 1
        # modification 4
        x[39] + x[230] + x[139] + x[164] == 1
        # modification 5
        sum(x[i] for i in K) <= 3
    end)

Academic license - for non-commercial use only


In [9]:
optimize!(Model_modify)
print("Optimal value: \$")
println(objective_value(Model_modify))
print("node ")
for i in I
    if value(x[i]) > 0.5
        print(i," ")
    end
end
print("are selected as distribution centers")

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 58324 rows, 58323 columns and 174488 nonzeros
Model fingerprint: 0xdd1c298b
Variable types: 0 continuous, 58323 integer (58323 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+09]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 0 rows and 1 columns
Presolve time: 0.42s
Presolved: 58324 rows, 58322 columns, 174488 nonzeros
Variable types: 0 continuous, 58322 integer (58322 binary)
Found heuristic solution: objective 8.298674e+08

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 2.671128e+08, 2063 iterations, 0.27 seconds

    Nodes    |    Current Node    